In [ ]:
import sys
import os
import pandas as pd
import json

In [ ]:
# Change the working directory to the project directory
project_path = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
os.chdir(project_path)

# Add the project directory to sys.path
sys.path.insert(0, project_path)

# Now you can import the print_path function from the tools.data_pipe module
from tools.folder_generator import create_folder
create_folder("localization_module")

# Data Cleaning

In [ ]:
path_list = []
module_path = os.path.abspath(os.path.join("localization_module"))
os.chdir(module_path)

folder_path = os.path.abspath(os.path.join("data","raw"))

for item in os.listdir(folder_path):
    if item.lower().endswith(".csv"):
        path_list.append(os.path.join(folder_path, item))

In [ ]:
name_list = [
    "localization2_gnss_pose",
    "localization2_localization_odom",
    "localization2_correct_pose_by_wall",
    "localization2_container_pose",
    "localization2_qc_match_pose",
    "localization2_optimized_pose",
    "localization2_magnetic_nail",
    "localization2_correct_pose_by_lane",
]

name = name_list[5]

def lmd_stamp(timestamp):
    """Timestamp alignment, all ending in even numbers."""
    if round(timestamp, 1) % 2 != 0: return round(timestamp, 1)-0.1
    else: return round(timestamp,1)

for ind in range(0, len(name_list)):
    name = name_list[ind]
    df_list = []
    for p in path_list:
        if name in p:
            print(p)
            df_list.append(pd.read_csv(p))
        else: pass

    df = pd.concat(df_list, axis=0)
    display((round(df["stamp"]-df["stamp"].shift(1), 1)).value_counts())

    df = df.sort_values(by='stamp')
    df["stamp"] = df["stamp"].apply(lambda x: lmd_stamp(x))

    processed_p = os.path.abspath(os.path.join('data', 'processed', f'{name}.csv'))
    df.to_csv(processed_p)

In [34]:
def get_info(folder_path):
    path_list, df_list = [], []
    num = 0

    for item in os.listdir(folder_path):
        if item.lower().endswith(".csv"):

            path_list.append(os.path.join(folder_path, item))

    for num in range(0,len(path_list)):
        p = path_list[num]
        df = pd.read_csv(p, index_col=0)
        print(num,
                os.path.basename(p), 
                # df.columns, 
                df.shape)

        if num == 0: 
            col = ["stamp", "container_pose_x", "container_pose_y","container_updated_pose_x", "container_updated_pose_y",]
        elif num == 1: col = ["stamp", "correct_pose_by_wall_x", "correct_pose_by_wall_y",]
        elif num == 2: col = ["stamp", "gnss_x", "gnss_y", "gnss_fsm_status"]
        elif num == 3: col = ["stamp", "base_footprint_x", "base_footprint_y",]
        elif num == 4: col = ["stamp", "optimized_pose_x", "optimized_pose_y",]
        elif num == 5: col = ["stamp", "qc_match_pose_x", "qc_match_pose_y","qc_match_updated_pose_x", "qc_match_updated_pose_y"]
        df = df[col].copy()

        df_list.append(df)
    return df_list
    
    
folder_path = os.path.abspath(os.path.join("data","processed"))
print(folder_path)
df_list = get_info(folder_path=folder_path)

c:\Users\westwell\Documents\.WorkDocuments\github\wwl_ds_projects\localization_module\data\processed
0 localization2_container_pose.csv (137066, 17)
1 localization2_correct_pose_by_wall.csv (325756, 7)
2 localization2_gnss_pose.csv (131155, 16)
3 localization2_localization_odom.csv (88538, 34)
4 localization2_optimized_pose.csv (9037, 4)
5 localization2_qc_match_pose.csv (63586, 17)


In [35]:
gps1 = pd.merge(df_list[2], df_list[0], how="left",on="stamp")
gps2 = pd.merge(gps1, df_list[1],how="left",on="stamp" )
gps3 = pd.merge(gps2, df_list[3],how="left",on="stamp" )
gps4 = pd.merge(gps3, df_list[4],how="left",on="stamp" )
gps5 = pd.merge(gps4, df_list[5],how="left",on="stamp" )

print(gps5.shape)

(132201, 18)


In [36]:
gps5.head()
display(gps5.columns)
export_p1 = os.path.abspath(os.path.join("data", "export", 'localization_horizontal_merge.csv'))
gps5.to_csv(export_p1)

Index(['stamp', 'gnss_x', 'gnss_y', 'gnss_fsm_status', 'container_pose_x',
       'container_pose_y', 'container_updated_pose_x',
       'container_updated_pose_y', 'correct_pose_by_wall_x',
       'correct_pose_by_wall_y', 'base_footprint_x', 'base_footprint_y',
       'optimized_pose_x', 'optimized_pose_y', 'qc_match_pose_x',
       'qc_match_pose_y', 'qc_match_updated_pose_x',
       'qc_match_updated_pose_y'],
      dtype='object')

In [37]:
def melt_data(df):
    # Identify columns that end with _x and _y
    x_cols = [col for col in gps5.columns if col.endswith("_x")]
    y_cols = [col for col in gps5.columns if col.endswith("_y")]

    # Create a list of tuples (x_col, y_col)
    xy_cols = list(zip(x_cols, y_cols))

    # Initialize the result DataFrame
    result = pd.DataFrame()

    for x_col, y_col in xy_cols:
        src = x_col.rsplit("_", 1)[0]  # Extract the source by removing the last "_x"
        temp_df = gps5[["stamp", x_col, y_col, "gnss_fsm_status"]].copy()
        temp_df.columns = ["stamp", "x", "y", "gnss_fsm_status"]
        temp_df["src"] = src
        result = pd.concat([result, temp_df], ignore_index=True)

    # Reorder the columns as desired
    result = result[["stamp", "x", "y", "src", "gnss_fsm_status"]]

    return result

res = melt_data(gps5)

print(res)

export_p2 = os.path.abspath(os.path.join("data", "export", 'localization_horizontal_melt_res.csv'))
res.to_csv(export_p2)


                stamp            x           y                    src  \
0        1.716181e+09  1001.401664  565.097769                   gnss   
1        1.716181e+09  1001.401511  565.097814                   gnss   
2        1.716181e+09  1001.401618  565.097958                   gnss   
3        1.716181e+09  1001.401760  565.098114                   gnss   
4        1.716181e+09  1001.401928  565.098294                   gnss   
...               ...          ...         ...                    ...   
1057603  1.716208e+09          NaN         NaN  qc_match_updated_pose   
1057604  1.716208e+09          NaN         NaN  qc_match_updated_pose   
1057605  1.716208e+09          NaN         NaN  qc_match_updated_pose   
1057606  1.716208e+09          NaN         NaN  qc_match_updated_pose   
1057607           NaN          NaN         NaN  qc_match_updated_pose   

         gnss_fsm_status  
0                    1.0  
1                    1.0  
2                    1.0  
3              

In [ ]:
# gnss -< gps -> 0,2,4,6,8 # gnss status
# container pose
# correct pose by wall
# base footprint 
# optimized
# qc match pose

# Feature Eng

### Location Section

In [38]:
cfg_pa = os.path.abspath(os.path.join('map_cfg.json'))
with open(cfg_pa, "r") as f:
    map_config = json.load(f)["MAP"]["YARD"]
print(map_config)

def lmd_section_tag(x, y):
    if map_config["x_min"] <= x <= map_config["x_max"] and map_config["y_min"] <= y <= map_config["y_max"]:
        return "YARD"
    else: return "QC"

res["YARD_or_QC"] = res.apply(lambda x: lmd_section_tag(x["x"], x["y"]), axis=1)

res.to_csv(export_p2)

{'x_min': 880, 'x_max': 1864, 'y_min': 243, 'y_max': 530}


### GPS not avaliable

In [ ]:
df = res[res["gnss_fsm_status"]==0].copy()
print(res.shape, df.shape)

gps5[gps5["gnss_fsm_status"]==0]["base_footprint_y"].isna().value_counts()
print(gps5.shape)

base_loc = gps5[gps5['gnss_fsm_status']==0]
print(base_loc.shape)
print(gps5.columns)

### Euclidean Distance